# JN-pipeline - Cell Nuclei Segmentation

The present JN shows an automatic pipeline to fetch data from OMERO, perform several processing steps and push back the results OMERO adding potential metadata and Region of Interest (ROIs).

## 1) Import Packages

In [ ]:
import getpass

import ezomero
import pandas as pd
import tifffile
import numpy as np

from skimage import filters, morphology, measure
from skimage.filters import gaussian
from skimage.color import label2rgb
from matplotlib import pyplot as plt
from microfilm.microplot import microshow

## 2) Set Parameters to login into OMERO

Add here your username or password. Adjust the OMERO host and port accordingly.

In [ ]:
OMEROUSER = input(f"Enter username: \t")
OMEROPASS = getpass.getpass(prompt = f"Enter password: \t")

OMEROHOST = ""
OMEROPORT = ""

## 3) Connect to OMERO

In [ ]:
# Connection Check:
conn=ezomero.connect(OMEROUSER, OMEROPASS, "", host=OMEROHOST, port=OMEROPORT, secure=True)

## Information about the connection and its status
print(conn.isConnected())
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

## 4) Browse your OMERO project folder

Browse your project and have an overview on the project and their content

In [ ]:
proj_ids = ezomero.get_project_ids(conn)
print("Current Project IDs: {proj_ids}".format(proj_ids=proj_ids))

ds_ids = ezomero.get_dataset_ids(conn, project=33)
print("Current Dataset IDs: {ds_ids}".format(ds_ids=ds_ids))

im_ids = ezomero.get_image_ids(conn, dataset=215)
im_ids = ezomero.filter_by_filename(conn, im_ids, "xxx")
print(im_ids)

### 4.1) Select your image

In [ ]:
img_id = input(f"Enter the image ID: \t")

img = conn.getObject("Image", img_id)

### 4.2) Get information about your image

In [ ]:
pixels = img.getPrimaryPixels()
print(img.getName(), img.getDescription())
# Retrieve information about an image.
print (" X:", img.getSizeX())
print (" Y:", img.getSizeY())
print (" Z:", img.getSizeZ())
print (" C:", img.getSizeC())
print (" T:", img.getSizeT())
# List Channels (loads the Rendering settings to get channel colors)
for channel in img.getChannels():
    print ('Channel:', channel.getLabel(),)
    print ('Color:', channel.getColor().getRGB())
    print ('Lookup table:', channel.getLut())


### 4.3) Plot your images

In [ ]:
## get nuclei (channel 0) and cytoplasm (channel 1)
nuclei = pixels.getPlane(0, 0, 0) 
cyto = pixels.getPlane(0, 1, 0) 

## recreate the original image
multi_channel_image = np.dstack((nuclei, cyto))
original_image = multi_channel_image.transpose((2, 0, 1))

fig, axs = plt.subplots(1,3, figsize=(10,10))
microshow(original_image, ax=axs[0], label_text= f'{img.getName()}', label_font_size=10, label_color='white')
microshow(nuclei, ax=axs[1], label_text=f'{img.getName()} - Nuclei', label_font_size=10, label_color='white')
microshow(cyto, ax=axs[2], label_text=f'{img.getName()} - Cytoplasm', label_font_size=10, label_color='white')

## 5) Image Processing 1 - Denoising

Remove background

In [ ]:
background_nuclei = gaussian(nuclei, sigma=100, preserve_range=True)
denoised_nuclei = nuclei - background_nuclei
fig, axs = plt.subplots(1, 2, figsize=(10,10))

microshow(nuclei, ax=axs[0], label_text='Nuclei - Original', label_color='white')
microshow(denoised_nuclei, ax=axs[1], label_text='Nuclei - Processed', label_color='white')

Apply a gaussian filter to denoise

In [ ]:
nuclei_gaussian = filters.gaussian(denoised_nuclei, sigma=2)

fig, axs = plt.subplots(1,2, figsize=(10,10))
microshow(nuclei, ax=axs[0], label_text='Nuclei - Original', label_color='white')
microshow(nuclei_gaussian, ax=axs[1], label_text='Nuclei - Processed', label_color='white')

## 6) Image Processing 2 - Binarization and Labelling

Apply a OTSU threshold and create the labelling

In [ ]:
# create a square with width and height = 3
square = morphology.square(5) 
square

threshold = filters.threshold_otsu(nuclei_gaussian)
binary_image = nuclei_gaussian >= threshold
opened = morphology.binary_opening(binary_image, square)
labeled_blobs = measure.label(opened)
cleaned = morphology.remove_small_objects(labeled_blobs, min_size=50)

fig, axs = plt.subplots(1,3, figsize=(10,10))

microshow(nuclei_gaussian, ax=axs[0], label_text='Original', label_color='white')
microshow(opened, ax=axs[1], label_text='Binary', label_color='white')
microshow(cleaned, ax=axs[2], label_text='Labels', label_color='white')

Overlay the labels and save the image

In [ ]:
image_label_overlay = label2rgb(cleaned, image=nuclei, bg_label=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(image_label_overlay)

tifffile.imwrite('test.tif', image_label_overlay, 
                 dtype=image_label_overlay.dtype, 
                 shape=image_label_overlay.shape)

## 7) Statistic and extract the table with features

In [ ]:
properties = measure.regionprops(cleaned, intensity_image=nuclei)
statistics = {
    'area':       [p.area               for p in properties],
    'mean':       [p.mean_intensity     for p in properties],
    'major_axis': [p.major_axis_length  for p in properties],
    'minor_axis': [p.minor_axis_length for p in properties]
}
df = pd.DataFrame(statistics)
df

In [ ]:
df.to_csv("/home/massei/Documents/03_Courses/03_My_Courses/2024-jn-omero-pipeline/02_jupyter_notebook/feature_extraction.csv")

## 8) Push back the results to OMERO and add metadata plus other information

In [ ]:
## Create a new dataset
did = ezomero.post_dataset(conn, "Processed", project_id=3799)
## push the image with REMBI annotation for analyzed data
ezomero.ezimport(conn, 'test.tif', dataset= did , ann= {"Analysis Result Type" : "Processed", 
                                                               "Data Used for The Analysis": f"{img.getName()}",
                                                                "Analysis method and details" : "Skimage"}, 
                                                                ns = "REMBI_Analyzed")

### 8.1) Add description and table with the results

In [ ]:
ezomero.put_description(conn, 'Image', 1, "This Image Was Created")
ezomero.post_table(conn, df, "Image", 1, title='Features Results', headers=True)

## 9) Close Connection

In [ ]:
conn.clone()

## 10) OPTIONAL - Add masks as ROIs

Add the labelling directly as ROIs in OMERO

In [ ]:
import omero_rois

In [ ]:
updateService = conn.getUpdateService()

def create_roi(img, shapes):
    roi = omero.model.RoiI()
    roi.setImage(img._obj)
    for shape in shapes:
        roi.addShape(shape)
    updateService.saveObject(roi)

mask = omero_rois.mask_from_binary_image(opened, rgba=(255, 0, 0, 50), z=0, c=0, t=0,
                                                     text="Labelling Masks", raise_on_no_mask=False)
create_roi(img, [mask])

# Automatize the pipeline trought a dataset on OMERO

In [ ]:
dataset = conn.getObject("Dataset", 1)

ID =[]

for images in dataset.listChildren():
    ID.append(images.getId())

print(ID)

for id in ID:
    img = conn.getObject("Image", id)
    ### repeat the pipeline ###